In [2]:
pip install dask[dataframe]

     |████████████████████████████████| 136 kB 5.5 MB/s 


In [3]:
import pandas as pd
from sklearn import tree
import dask
import dask.dataframe as dd
from dask.distributed import Client, progress
import dask.array as da
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
import pandas as pd
import numpy as np

#prefs
pd.set_option('max_columns', None)

#testing
import doctest
import unittest

In [4]:
policy = pd.read_csv('/content/USA_policy_2022.csv')
policy.head(4)

,Unnamed: 0,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,C2_Flag,C2_Notes,C3_Cancel public events,C3_Flag,C3_Notes,C4_Restrictions on gatherings,C4_Flag,C4_Notes,C5_Close public transport,C5_Flag,C5_Notes,C6_Stay at home requirements,C6_Flag,C6_Notes,C7_Restrictions on internal movement,C7_Flag,C7_Notes,C8_International travel controls,C8_Notes,E1_Income support,E1_Flag,E1_Notes,E2_Debt/contract relief,E2_Notes,E3_Fiscal measures,E3_Notes,E4_International support,E4_Notes,H1_Public information campaigns,H1_Flag,H1_Notes,H2_Testing policy,H2_Notes,H3_Contact tracing,H3_Notes,H4_Emergency investment in healthcare,H4_Notes,H5_Investment in vaccines,H5_Notes,H6_Facial Coverings,H6_Flag,H6_Notes,H7_Vaccination policy,H7_Flag,H7_Notes,H8_Protection of elderly people,H8_Flag,H8_Notes,M1_Wildcard,M1_Notes,V1_Vaccine Prioritisation (summary),V1_Notes,V2A_Vaccine Availability (summary),V2_Notes,V2B_Vaccine age eligibility/availability age floor (general population summary),V2C_Vaccine age eligibility/availability age floor (at risk summary),V2D_Medically/ clinically vulnerable (Non-elderly),V2E_Education,V2F_Frontline workers (non healthcare),V2G_Frontline workers (healthcare),V3_Vaccine Financial Support (summary),V3_Notes,V4_Mandatory Vaccination (summary),V4_Notes,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,30784,United States,USA,NaN,NaN,NAT_TOTAL,20220101,2.0,0.0,NaN,2.0,0.0,NaN,1.0,0.0,NaN,4.0,0.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,4.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,5.0,1.0,NaN,2.0,0.0,NaN,NaN,NaN,2.0,NaN,3.0,NaN,5-15 yrs,5-15 yrs,2.0,2.0,2.0,2.0,5.0,NaN,1.0,NaN,55024111.0,824703.0,53.24,53.24,63.81,63.81,52.6,52.6,60.12,60.12,0.0,0.0
1,30785,United States,USA,NaN,NaN,NAT_TOTAL,20220102,2.0,0.0,NaN,2.0,0.0,NaN,1.0,0.0,NaN,4.0,0.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,4.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,5.0,1.0,NaN,2.0,0.0,NaN,NaN,NaN,2.0,NaN,3.0,NaN,5-15 yrs,5-15 yrs,2.0,2.0,2.0,2.0,5.0,NaN,1.0,NaN,55309989.0,825081.0,53.24,53.24,63.81,63.81,52.6,52.6,60.12,60.12,0.0,0.0
2,30786,United States,USA,NaN,NaN,NAT_TOTAL,20220103,2.0,0.0,NaN,2.0,0.0,NaN,1.0,0.0,NaN,4.0,0.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,4.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,5.0,1.0,NaN,2.0,0.0,NaN,NaN,NaN,2.0,NaN,3.0,NaN,5-15 yrs,5-15 yrs,2.0,2.0,2.0,2.0,5.0,NaN,1.0,NaN,56354828.0,826921.0,53.24,53.24,63.81,63.81,52.6,52.6,60.12,60.12,0.0,0.0
3,30787,United States,USA,NaN,NaN,NAT_TOTAL,20220104,2.0,0.0,Several school districts across the country ar...,2.0,0.0,NaN,1.0,0.0,NaN,4.0,0.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,4.0,"On December 31st, the Biden administration lif...",0.0,NaN,Per the USA government guidance on employment ...,0.0,"On August 26th, the Supreme Court ruled the CD...",NaN,NaN,NaN,NaN,2.0,1.0,The CDC still maintains a robust public inform...,3.0,Testing is generally available across the US. ...,1.0,Although the highest level of contact tracing ...,NaN,NaN,NaN,NaN,3.0,0.0,NaN,5.0,1.0,Vaccines are universally available across the ...,2.0,0.0,NaN,NaN,NaN,2.0,NaN,3.0,NaN,5-15 yrs,5-15 yrs,2.0,2.0,2.0,2.0,5.0,NaN,1.0,NaN,57166940.0,829230.0,53.24,53.24,63.81,63.81,52.6,52.6,60.12,60.12,0.0,0.0


In [5]:
#policy = policy.filter(["CountryName", "Date", "ConfirmedCases", "ConfirmedDeaths", "StringencyIndex", "StringencyIndexForDisplay", "StringencyLegacyIndex", "StringencyLegacyIndexForDisplay", "GovernmentResponseIndex", "GovernmentResponseIndexForDisplay	", "ContainmentHealthIndex", "ContainmentHealthIndexForDisplay", "EconomicSupportIndex", "EconomicSupportIndexForDisplay"])
policy = policy.filter(["Date","RegionName", "ConfirmedCases", "ConfirmedDeaths", "StringencyIndex"])
policy

,Date,RegionName,ConfirmedCases,ConfirmedDeaths,StringencyIndex
0,20220101,NaN,55024111.0,824703.0,53.24
1,20220102,NaN,55309989.0,825081.0,53.24
2,20220103,NaN,56354828.0,826921.0,53.24
3,20220104,NaN,57166940.0,829230.0,53.24
4,20220105,NaN,57829451.0,831412.0,53.24
...,...,...,...,...,...
5403,20220410,Wyoming,156281.0,1796.0,NaN
5404,20220411,Wyoming,156281.0,1796.0,NaN
5405,20220412,Wyoming,156258.0,1801.0,NaN
5406,20220413,Wyoming,NaN,NaN,NaN


In [6]:
data = pd.read_csv('/content/us-counties-2022.csv')
data.head(5)

,date,county,state,fips,cases,deaths
0,2022-01-01,Autauga,Alabama,1001.0,11018,160.0
1,2022-01-01,Baldwin,Alabama,1003.0,39911,593.0
2,2022-01-01,Barbour,Alabama,1005.0,3860,81.0
3,2022-01-01,Bibb,Alabama,1007.0,4533,95.0
4,2022-01-01,Blount,Alabama,1009.0,11256,198.0


In [7]:
data = data.filter(["date", "state", "fips", "cases", "deaths"])
data.head(4)

,date,state,fips,cases,deaths
0,2022-01-01,Alabama,1001.0,11018,160.0
1,2022-01-01,Alabama,1003.0,39911,593.0
2,2022-01-01,Alabama,1005.0,3860,81.0
3,2022-01-01,Alabama,1007.0,4533,95.0


In [8]:
#convert types
data["date"] = dd.to_datetime(data.date)
data.head()

,date,state,fips,cases,deaths
0,2022-01-01,Alabama,1001.0,11018,160.0
1,2022-01-01,Alabama,1003.0,39911,593.0
2,2022-01-01,Alabama,1005.0,3860,81.0
3,2022-01-01,Alabama,1007.0,4533,95.0
4,2022-01-01,Alabama,1009.0,11256,198.0


In [9]:
#drop the column with Nan
df = data.dropna()
df

,date,state,fips,cases,deaths
0,2022-01-01,Alabama,1001.0,11018,160.0
1,2022-01-01,Alabama,1003.0,39911,593.0
2,2022-01-01,Alabama,1005.0,3860,81.0
3,2022-01-01,Alabama,1007.0,4533,95.0
4,2022-01-01,Alabama,1009.0,11256,198.0
...,...,...,...,...,...
426201,2022-05-11,Wyoming,56037.0,11088,126.0
426202,2022-05-11,Wyoming,56039.0,10074,16.0
426203,2022-05-11,Wyoming,56041.0,5643,39.0
426204,2022-05-11,Wyoming,56043.0,2358,44.0


In [10]:
#drop the column with Nan
df1 = policy.dropna()
df1

,Date,RegionName,ConfirmedCases,ConfirmedDeaths,StringencyIndex
104,20220101,Alaska,157169.0,978.0,45.83
105,20220102,Alaska,157169.0,978.0,45.83
106,20220103,Alaska,160089.0,979.0,45.83
107,20220104,Alaska,160089.0,979.0,45.83
108,20220105,Alaska,161678.0,979.0,45.83
...,...,...,...,...,...
5394,20220401,Wyoming,156167.0,1791.0,39.81
5395,20220402,Wyoming,156167.0,1791.0,39.81
5396,20220403,Wyoming,156167.0,1791.0,39.81
5397,20220404,Wyoming,156167.0,1791.0,39.81


In [27]:

X = df["cases"]
Y = df["state"]



In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train ,y_test = train_test_split(X, Y, test_size=0.2) # 切分数据集,分成训练集和验证集
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [14]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()
lg.fit(x_train, y_train)
score = lg.score(x_test, y_test)

ValueError: ignored